<a href="https://colab.research.google.com/github/PeterNaggschga/Letter-Variations-in-First-Names-IS/blob/main/Report_Gruppe_05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Version: 2023.01.09

---

# Report Letter Variations in First Names

##Aufgabenstellung
In unserem Projekt haben wir uns mit Buchstabenvariationen in Vornamen befasst. Unsere Aufgabe war es, Vornamen aus einem Datensatz zu extrahieren und diese zu gruppieren. Zusätzlich sollten wir daraus eine Substitutionsmatrix ableiten, die die Häufigkeit darstellt, mit der ein Buchstabe durch einen anderen Buchstaben ersetzt wird.



#Projektbeschreibung
##Datensatz
Unser Projekt arbeitet mit einem Datensatz, der auf PubMed basiert. Dafür nutzen wir Paper, die im Januar 2022 veröffentlicht wurden und extrahieren anschließend die Vornamen der jeweiligen Autoren.

##Namensextraktion & Data Cleaning
Für die Namensextraktion ließen wir uns zuerst nur mit Regulären Ausdrücken, die Namen der Autoren der Papers aus unserem Datensatz ausgeben. Im nächsten Schritt teilten wir Nach- und Vorname die i.d.R. durch ein Komma getrennt sind und speichern uns nur die Vornamen.

Diese Methode der Namensextraktion war jedoch noch nicht ganz ausgereift. Beispielsweise waren auch Vornamen wie "G-R" dabei. Der überarbeitete Ansatz, Namen zu Extrahieren, funktioniert folgendermaßen:

Zuerst lassen wir uns mit einer spezifischen Anfrage und einer Obergrenze eine Menge an Papern zurückgeben. Die Funktion "getPapers(myQuery, maxPapers, myEmail)" verwendet dafür die Bio-Bibliothek und insbesondere die Unterbibliotheken Entrez und Medline, um Paper von PubMed zu suchen und zu erhalten. Zurückgegeben wird eine Liste von Papern als Liste von Dictionaries. Diese enthalten Zusammenfassung, Autoren usw.

Die Funktion "retrieveAllFirstNames(records)" nimmt eine Liste von Papern entgegen und extrahiert die Vornamen der Autoren (als kombinierte Liste) mit der split-Methode. Dabei nutzen wir aus, dass Vornamen und Nachnamen durch ein Komma, und Vornamen untereinander mit einem Leerzeichen, getrennt sind. Anschließend gibt die Funktion eine Liste von Vornamen als Strings zurück. 

Der "Data-Cleaning-Schritt" findet ebenfalls in der Funktion "retrieveAllFirstNames" statt. Um vorhin genanntes Problem zu vermeiden, werden zu kurze Namen oder Namen mit bestimmten Zeichen entfernt. Dabei filtern wir besonders Namen, welche Zahlen enthalten (z. B. "2nd"), abgekürzte Namen (z. B. "P.") und Strings, welche "-" enthalten. Mit letzterem werden zusammenhängende Namen und die Abkürzungen wie z. B. "P-G" entfernt. Zum Schluss wenden wir noch die string.lower-Methode an, um Großbuchstaben mit Kleinbuchstaben zu ersetzen.

##Clustering
Nachdem die Namenextraktion abgeschlossen ist, müssen die Namen "gruppiert" bzw. "geclustert" werden. Dafür haben wir uns an der DBSCAN-Methode orientiert. Unsere Implementierung der DBSCAN-Methode verwendet eine Radix Tree-Datenstruktur, um Wörter zu speichern und ähnliche Wörter schnell zu finden. Der Algorithmus nutzt nur Namen mit Länge min_word_length oder länger. Alle Namen werden in Kleinbuchstaben umgewandelt und in einer Radix Tree-Datenstruktur gespeichert.

Der Algorithmus iteriert dann durch jeden Vornamen und betrachtet ihn als Kernpunkt eines Clusters, wenn er min_samples Nachbarn besitzt (er selbst zählt dazu). Dafür werden alle Namen gesucht, die innerhalb eines Bereichs (Hamming-Distanz von maximal 1) um den aktuellen Namen liegen. Bereits erkundete Namen/Nachbarn zählen dabei nicht mehr. Ist ein Name ein Kernknoten, so werden umliegende Namen dem Cluster hinzugefügt und der Algorithmus wird rekursiv auf jeden dieser Namen angewendet, um alle Namen zu finden, die noch Teil des Clusters sind. Wenn die Anzahl der Namen, die innerhalb des zuvor definierten Bereichs liegen, kleiner als min_samples ist, wird der aktuelle Name als Randpunkt betrachtet und umliegende Knoten nicht dem Cluster hinzugefügt. 
Wenn der Algorithmus einen Cluster gefunden hat, der zu groß ist (größer als 600 Namen), wird der Algorithmus rekursiv auf diesen Cluster angewendet und versucht, ihn in kleinere Cluster aufzuteilen. Dazu wird min_samples erhöht. Wenn der Algorithmus einen oder mehrere Cluster gefunden hat, die länger als ein Element sind und kleiner oder gleich max_cluster_length sind, werden sie zur Liste der Clusters hinzugefügt. 
Der Algorithmus wird fortgesetzt, bis alle Namen im Datensatz entweder einem Cluster zugeordnet wurden oder als Rauschen betrachtet wurden (d.h. sie können keinem Cluster zugeordnet werden). Wenn keine Namen mehr in der Liste verbleiben, wird die Liste der Clusters zurückgegeben.

##RadixTree
Die RadixTree-Datenstruktur nutzen wir, wie schon erwähnt, um die Vornamen zu speichern und effizient ähnliche Namen auslesen zu können. Unsere Implementierung des RadixTrees verwendet eine rekursive Datenstruktur, die aus Knoten besteht, die durch Übergänge miteinander verbunden sind. Jeder Knoten speichert eine Liste von Übergängen als Schlüssel-Wert-Paare in einem Dictionary. Der Wert des Schlüssel-Wert-Paares ist ein weiterer Knoten und der Schlüssel ist der Übergang zwischen den beiden Knoten. Wenn ein Knoten ein Blatt bzw. das Ende eines Wortes darstellt, ist die isWord-Variable des Knotens auf "True" gesetzt.

Die insertWord-Methode wird verwendet, um ein Wort in den Radix Tree einzufügen. Der Algorithmus geht durch das Wort von links nach rechts und prüft, ob der aktuelle Übergang bereits im Tree vorhanden ist. Wenn ja, wird der Algorithmus rekursiv auf den nächsten Teil des Wortes angewendet, um ihn in den Unterbaum einzufügen. Ist der Übergang nicht vorhanden, wird er hinzugefügt.

Die strRecursive-Methode wird verwendet, um eine lesbare Darstellung des Radix Trees zu generieren.

RadixTreesByWordLength ist eine Hilfsklasse bzw. ein Container mit einem Dictionary, das Verweise auf mehrere RadixTrees speichert, die Wörtern mit gleicher Länge speichern.

Die getSimilarWordsOfSameLength-Methode wird verwendet, um ein RadixTree zu suchen, der Vornamen der geforderten Länge speichert. Wird so einer gefunden, wird die Methode "getSimilarWordsOfSameLength" des entsprechenden RadixTrees aufgerufen. Diese findet letztendlich alle Wörter in diesem Tree (der Wörter gleicher Länge speichert), die weniger oder gleich viele unterschiedliche Buchstaben haben als in maximumDifferentLetters festgelegt wurden.

##Substitutionsmatrix
Unsere Implementation der Substitutionsmatrix ist an die Funktionsweise von BLOSUM angelehnt. Normalerweise nutzt man BLOSUM für das Sequenzalignement von Proteinen, bei uns dient sie aber dazu, aus einer Liste von Namen und einer Liste von Clustern ähnlicher Namen eine Substitutionsmatrix zu berechnen. Typischerweise wäre es so, dass, wenn zwei Namen verglichen werden, bewertet wird, ob die Buchstaben an einer bestimmten Stelle gleich sind. Sind sie gleich, wird ein Score von typischerweise +1 oder +5 zugewiesen, und wenn sie ungleich sind, wird ein Score von -1 oder -4 zugewiesen. Unsere Substitutionsmatrix gibt letztendlich an, wie wahrscheinlich es ist, dass ein Buchstabe durch einen anderen ersetzt wird.

Die Namen und Cluster werden als Eingabe in den Konstruktor der Klasse gegeben, der die Instanzvariablen initialisiert und dann mehrere Methoden zur Berechnung der Substitutionsmatrix aufruft. Die Methode "calc_name_occurrences" berechnet die Anzahl der Vorkommen jedes Namens in der Liste der Namen. Die Methode "calc_letter_prob" berechnet die Auftrittswahrscheinlichkeit jedes Buchstabens, der in einem Namen vorkommt. Die Methode "calc_sub_probs" berechnet die Wahrscheinlichkeit, dass ein Buchstabe durch einen anderen Buchstaben ersetzt wird. Die Methode "calc_substitution_matrix" berechnet schließlich die Substitutionsmatrix selbst, indem sie die in "calc_sub_probs" berechneten Wahrscheinlichkeiten mit den als Eingabe angegebenen Skalierungsfaktoren kombiniert.

---

Dabei werden die Matrixeinträge nach folgender Formel berechnet: 

$$ S_{i,j} = \frac{1}{\lambda_2} \log_2{(\lambda_1 \cdot \frac{p_{ij}}{q_i \cdot q_j})} $$

$ p_{ij} $ ... Wahrscheinlichkeit, dass (innerhalb eines Clusters) i durch j ersetzt wird \\
$ q_i, q_j $ ... Wahrscheinlichkeiten, dass i bzw. j in einem Wort auftritt \\
$ \lambda_i $ ... (frei wählbare) Skalierungsfaktoren \\
Dabei wird $ \lambda_1 $ genutzt, um die Werte in der Matrix in positive oder negative Richtung zu verschieben. 
$ \lambda_2 $ skaliert größe der Werte linear. 
Die Funktion SubstitutionMatrix.autoscale() skaliert die Werte der Matrix standardmäßig so, dass für alle Werte gilt $ S_{i,j} \in [-5, 5] $ (Abweichungen von bis zu 0.1% möglich).

---

Die Substitutionswahrscheinlichkeit $p_{ij}$ wird iterativ für jedes Cluster $C$ berechnet. Dabei werden alle Wörter des Clusters zeichenweise an den Stellen $k=1,...,l_C$ betrachtet, wobei $l_C$ die Länge der Wörter im Cluster $C$ angibt. Nun wird für jedes Buchstabenpaar $(i, j)$ die Wahrscheinlichkeit des gemeinsamen Auftretens in $C$ an Stelle $k$ wie folgt berechnet:

$$ p_{ij, k}^{(C)} = \frac{f_k^{(C)}(i, j)}{\displaystyle \sum_{a = 1} ^{n} \sum_{b = a} ^{n} f_k^{(C)}(a, b)} $$

$n$ ... Anzahl Buchstaben im betrachteten Alphabet (bei uns gilt n = 26)

Dabei gilt für die Anzahl möglicher Buchstabenpaare $(i, j)$ in Cluster $C$ an Stelle $k$:

Für $i \neq j$ gilt: 
$$ f_k^{(C)}(i, j) = o_{k, i}^{(C)} \cdot o_{k, j}^{(C)} $$

sonst: 
$$ f_k^{(C)}(i, i) = \frac{o_{k, i}^{(C)} \cdot (o_{k, i}^{(C)} - 1)}{2} $$

wobei $o_{k, i}^{(C)}$ angibt, wie oft im Cluster $C$ an Stelle $k$ der Buchstabe an Stelle $i$ des Alphabets vorkommt.

---

Die so erhaltenen Substitutionswahrscheinlichkeit $p_{ij, k}^{(C)}$ werden nun aufaddiert. Dabei wird nach der Anzahl der entsprechenden Substitutionen $o_{k, i}^{(C)}$ gewichtet:

$$ p_{ij} = \frac{1}{o_{ij}} \displaystyle \sum_{C \in M} \sum_{k=1}^n ((o_{k, i}^{(C)} + o_{k, j}^{(C)}) \cdot p_{ij, k}^{(C)} ) $$

wobei $M$ die Menge aller Cluster und $o_{ij}$ die Gesamtzahl der gemeinsamen Vorkommen von $i$ und $j$ darstellt.

---

Die Methode set_names überschreibt die Liste von Namen und die Clustering-Liste (falls angegeben) und berechnet dann eine neue Substitutionsmatrix.

Die Methode "set_clusters" setzt eine neue Clustering-Liste und berechnet dann eine neue Substitutionsmatrix.

Die Methode "set_scaling" setzt neue Skalierungsfaktoren und berechnet dann eine neue Substitutionsmatrix.

Die Methode "__ str__" gibt eine String-Repräsentation der Substitutionsmatrix zurück.


##Demo
Für unser Projekt haben wir eine kleine Demo zum Testen angefertigt, die die oben beschriebenen Funktionalitäten nutzt. Dafür lassen wir uns Paper vom 20. Januar 2021 aus der PubMed-Datenbank ausgeben. Im Anschluss extrahieren wir alle Vornamen, aus unserem Datensatz. Diese Vornamen Clustern wir und übergeben letztendlich alle Namen und Cluster an die Substitutionsmatrix. 